# Gemma 3n multimodal

Test de funcionamiento de multimodalidad en gemma 3n
- System promt experto en productividad
- Test nota de texto
- Test nota de audio
- Test nota de video
- Simulación de test de distintas notas a lo largo de un dia y resumen del dia.

## Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [2]:
%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

## FastModel (Unsloth)

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [3]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None, # None for auto detection
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.11: Fast Gemma3N patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

## Data for testing

Let's first experience how Gemma 3N can handle multimodal inputs. We use Gemma 3N's recommended settings of `temperature = 1.0, top_p = 0.95, top_k = 64`

In [11]:
from transformers import TextStreamer
# Helper function for inference
def do_gemma_3n_inference(messages, max_new_tokens = 128):
    _ = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt = True, # Must add for generation
            tokenize = True,
            return_dict = True,
            return_tensors = "pt",
        ).to("cuda"),
        max_new_tokens = max_new_tokens,
        temperature = 1.0, top_p = 0.95, top_k = 64,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

- Nota de texto:
9:30 Estoy de vacaciones en Gran Canaria hoy me quedo en las palmas
- Playa
10:30 <img src="https://www.grancanaria.com/turismo/fileadmin/_processed_/csm_Las_Canteras_02_7cac71214f.jpg">

Hemos encontrado buen sitio en la playa

- Comida
14:00<img src="https://i.blogs.es/8c3360/pollo_curry/1366_2000.jpg">

- Tarde
17:00<img src="https://cdn.getyourguide.com/image/format=auto,fit=crop,gravity=auto,quality=60,width=620,height=400,dpr=2/tour_img/893c6d0e835705f5deade88c7b398d2c3bc039c28ffe13cf83ac14682953b41d.jpg">


23:00
https://drive.google.com/file/d/1UPObDJUB8N84IaypYqcFmT_HyDZTleux/view?usp=sharing



In [7]:
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1UPObDJUB8N84IaypYqcFmT_HyDZTleux" -O audio.wav

--2025-07-31 16:37:40--  https://drive.google.com/uc?export=download&id=1UPObDJUB8N84IaypYqcFmT_HyDZTleux
Resolving drive.google.com (drive.google.com)... 142.251.12.100, 142.251.12.101, 142.251.12.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.12.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1UPObDJUB8N84IaypYqcFmT_HyDZTleux&export=download [following]
--2025-07-31 16:37:40--  https://drive.usercontent.google.com/download?id=1UPObDJUB8N84IaypYqcFmT_HyDZTleux&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.12.132, 2404:6800:4003:c11::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.12.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 476320 (465K) [audio/wav]
Saving to: ‘audio.wav’

audio.wav           100%[===================>] 465.16K  --.-KB/s    in 0.006s  

2025

In [9]:
from IPython.display import Audio, display
Audio(filename:='audio.wav')

# Test

## Test input test

In [12]:
messages = [{
    "role" : "user",
    "content": [
        { "type": "text", "text" : "9:30 Estoy de vacaciones en Gran Canaria hoy me quedo en las palmas" },
        { "type": "text",  "text" : "¿Dónde estoy de vacaciones?" }
    ]
}]
# You might have to wait 1 minute for Unsloth's auto compiler
do_gemma_3n_inference(messages, max_new_tokens = 256)

Si estás de vacaciones en Gran Canaria y te quedas en Las Palmas, ¡estás de vacaciones en **Gran Canaria**! 

Las Palmas es la capital de Gran Canaria, por lo tanto, aunque te alojes allí, sigues disfrutando de tus vacaciones en la isla. 😊
<end_of_turn>


## Test input image

In [16]:
image_link = "https://www.grancanaria.com/turismo/fileadmin/_processed_/csm_Las_Canteras_02_7cac71214f.jpg"
messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : image_link },
        { "type": "text", "text" : "Playa 10:30" },
        { "type": "text",  "text" : "¿había mucha gente en la playa cuando fui?" },
        { "type": "text",  "text" : "¿A qué hora fui a la playa?" }
    ]
}]
# You might have to wait 1 minute for Unsloth's auto compiler
do_gemma_3n_inference(messages, max_new_tokens = 256)

Según la imagen, parece que **no había mucha gente en la playa** cuando se tomó la foto. Hay algunas personas dispersas, pero la playa no parece estar muy concurrida.

La hora en que se tomó la foto es **10:30**.<end_of_turn>


## Test Audio

⚠️ Audio in wav is not working properly and mp3 is ok.

In [24]:
audio_file = "download.mp3"

messages = [{
    "role" : "user",
    "content": [
        { "type": "audio", "audio" : audio_file },
        { "type": "text",  "text" : "What is this audio about?" }
    ]
}]
do_gemma_3n_inference(messages, max_new_tokens = 256)

The audio is about someone's vacation in Gran Canaria. They went to the beach in the morning, had lunch, and then took surfing lessons in the afternoon. The weather started out good but then it deteriorated.<end_of_turn>


# Day summary
Let's combine different data types to see if the model can summarize a daily journal.

In [33]:
food_image_link = "https://i.blogs.es/8c3360/pollo_curry/1366_2000.jpg"
surf_image = "https://cdn.getyourguide.com/image/format=auto,fit=crop,gravity=auto,quality=60,width=620,height=400,dpr=2/tour_img/893c6d0e835705f5deade88c7b398d2c3bc039c28ffe13cf83ac14682953b41d.jpg"
messages = [
    {
        "role": "system",
        "content": [
            { "type": "text", "text" :  "You are a journaling expert. Your job is to help the user make sense of their day through their spontaneous notes. Use a calm and reflective tone. If there's emotion, help name it. If there’s a pattern, highlight it. If it’s chaotic, help bring clarity. Do not overanalyze — guide gently." }
        ]
    },
    {
        "role": "user",
        "content": [
            { "type": "text", "text" : "9:30 Estoy de vacaciones en Gran Canaria hoy me quedo en las palmas" },
            { "type": "image", "image" : image_link },
            { "type": "text", "text" : "Comemos cerca de las canteras" },
            { "type": "image", "image" : food_image_link },
            { "type": "image", "image": surf_image},
            { "type": "audio", "audio" : audio_file },
            { "type": "text",  "text": "¿Puedes ayudarme a reflexionar sobre el día de hoy?" },
        ]
    },

]
do_gemma_3n_inference(messages, max_new_tokens = 512)

It sounds like you had a lovely day in Gran Canaria! It's wonderful to be on vacation and taking time for yourself. 

It seems like your day was filled with pleasant activities: starting with a morning at the beach, followed by a delicious meal, and then an afternoon of surfing lessons. It's great that the weather started out nice for surfing, even if it later changed. 

There's a sense of relaxation and enjoyment in your notes. It's nice to see you taking in the beauty of the island and making the most of your vacation. 

It seems like you're embracing the vacation vibe and enjoying the simple pleasures of sunshine, the beach, and good food. This is a great pattern to continue!<end_of_turn>
